# Digits

[_Digits_](https://www.nytimes.com/games/digits) is a daily numbers puzzle game from The New York Times.

You start with six nonnegative integers, and you are given a _target_, another nonnegative integer.

At each turn, you may replace any two numbers with their sum, difference, product or ratio. You may not subtract a greater number from a smaller one, nor divide a number by another that is not one of its divisors. In other words, you may only perform an operation if it results in a nonnegative integer.

You may perform any number of operations, and you may reuse the results of previous operations, but each operation "consumes" its operands. For example, if you start with $\{1, 2, 3\}$, you may replace $1$ and $2$ with $2 - 1 = 1$ and you would be left with $\{1, 3\}$. You may have multiple copies of the same number at the same time.

The goal of the game is to perform an operation which result is as close as possible to the target. Your score is the difference in absolute value between the target and the result of any operation you performed. The best possible score is zero, which you achieve when you hit the target exactly.

If you're thinking a computer should do this, you're not alone.

In [1]:
from itertools import combinations, product
from operator import add, sub, mul, floordiv
from dataclasses import dataclass
from typing import Iterable, List, Tuple, Callable


@dataclass
class operation:
    op: Callable[[int, int], int]
    valid: Callable[[int, int], bool]
    r: str
    
    def __call__(self, a, b):
        return self.op(a, b)
    
    def __repr__(self):
        return self.r
    
    def is_valid(self, a, b):
        return self.valid(a, b)
    

operations = [
    operation(add, lambda a, b: True, '+'),
    operation(sub, lambda a, b: a >= b, '-'),
    operation(mul, lambda a, b: True, '*'),
    operation(floordiv, lambda a, b: b != 0 and a % b == 0, '/')
]


def results(s: Iterable[int], target: int, o = operations) -> List[Tuple[int, str]]:
    state = [[(x, str(x)) for x in s]]
    off = set()
    while state:
        x = state.pop()
        for ((a, ax), (b, bx)), op in product(combinations(x, 2), o):
            if op.is_valid(a, b):
                t = op(a, b), f'({ax} {repr(op)} {bx})'
                l = x.copy()
                l.remove((a, ax))
                l.remove((b, bx))
                l.append(t)
                state.append(l)
                off.add(t)
                
    return sorted(off, key=lambda u: abs(target - u[0]))

Some examples:

In [2]:
R = results((3, 4, 6, 8, 9, 11), 234)

for n, expr in R:
    assert n == eval(expr)

len(R), R[:5]

(90736,
 [(234, '(6 * (4 + (11 + (3 * 8))))'),
  (234, '(9 * ((4 * 11) - (3 * 6)))'),
  (234, '((6 * 11) + (8 * (9 + (3 * 4))))'),
  (234, '((9 + (4 * (6 * 8))) + (3 * 11))'),
  (234, '(6 + (3 * (4 + (8 * 9))))')])

In [3]:
R = results({5, 7, 11, 19, 20, 23}, 476)

for n, expr in R:
    assert n == eval(expr)

len(R), R[:5]

(157548,
 [(476, '(((23 * 7) + (20 * 11)) + (19 * 5))'),
  (476, '((7 * (19 + (23 * 11))) / (20 / 5))'),
  (476, '(7 * ((20 - 5) + (19 + (23 + 11))))'),
  (476, '((19 - (20 - (7 + 11))) * (5 + 23))'),
  (476, '(7 * ((20 - 5) + (23 + (19 + 11))))')])